In [1]:
import kaolin
import torch
import os
import numpy as np
import matplotlib.pyplot as plt
import polyscope as ps

# import diffvoronoi
import pygdel3d
import sdfpred_utils.sdfpred_utils as su
import sdfpred_utils.loss_functions as lf
from pytorch3d.loss import chamfer_distance
from pytorch3d.ops import knn_points, knn_gather
import torch
from torch import nn

# cuda devices
device = torch.device("cuda:0")
print("Using device: ", torch.cuda.get_device_name(device))
torch.manual_seed(69)

input_dims = 3
lr_sites = 0.005
# lr_model = 0.00001
destination = "./images/autograd/End2End_DCCVT_interpolSDF/"
model_trained_it = ""

# mesh = ["sphere"]

mesh = ["gargoyle", "/home/wylliam/dev/Kyushu_experiments/data/gargoyle"]
trained_model_path = f"/home/wylliam/dev/HotSpot/log/3D/pc/HotSpot-all-2025-04-24-18-16-03/gargoyle/gargoyle/trained_models/model{model_trained_it}.pth"

# mesh = ["gargoyle_unconverged", "/home/wylliam/dev/Kyushu_experiments/mesh/gargoyle_unconverged"]
# trained_model_path = f"/home/wylliam/dev/HotSpot/log/3D/pc/HotSpot-all-2025-04-24-18-16-03/gargoyle/gargoyle/trained_models/model_2000.pth"


# mesh = ["chair", "/home/wylliam/dev/Kyushu_experiments/data/chair"]
# trained_model_path = f"/home/wylliam/dev/HotSpot/log/3D/pc/HotSpot-all-2025-05-02-17-56-25/chair/chair/trained_models/model{model_trained_it}.pth"

# mesh = ["bunny", "/home/wylliam/dev/Kyushu_experiments/data/bunny"]
# trained_model_path = f"/home/wylliam/dev/HotSpot/log/3D/pc/HotSpot-all-2025-04-25-17-32-49/bunny/bunny/trained_models/model{model_trained_it}.pth"


Using device:  NVIDIA GeForce RTX 3090


In [2]:
num_centroids = 32**3
grid = 32  # 128
print("Creating new sites")
noise_scale = 0.005
domain_limit = 1
x = torch.linspace(-domain_limit, domain_limit, int(round(num_centroids ** (1 / 3))))
y = torch.linspace(-domain_limit, domain_limit, int(round(num_centroids ** (1 / 3))))
z = torch.linspace(-domain_limit, domain_limit, int(round(num_centroids ** (1 / 3))))
meshgrid = torch.meshgrid(x, y, z)
meshgrid = torch.stack(meshgrid, dim=3).view(-1, 3)

# add noise to meshgrid
meshgrid += torch.randn_like(meshgrid) * noise_scale


sites = meshgrid.to(device, dtype=torch.float32).requires_grad_(True)

print("Sites shape: ", sites.shape)
print("Sites: ", sites[0])
ps.init()


Creating new sites
Sites shape:  torch.Size([32768, 3])
Sites:  tensor([-1.0027, -1.0065, -0.9978], device='cuda:0', grad_fn=<SelectBackward0>)


/home/wylliam/dev/Kyushu_experiments-1/venv/lib/python3.12/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 3.3.0 NVIDIA 575.64.03


In [3]:
# LOAD MODEL WITH HOTSPOT

import sys

if mesh[0] != "sphere":
    sys.path.append("3rdparty/HotSpot")
    from dataset import shape_3d
    import models.Net as Net

    loss_type = "igr_w_heat"
    loss_weights = [350, 0, 0, 1, 0, 0, 20]

    train_set = shape_3d.ReconDataset(
        file_path=mesh[1] + ".ply",
        n_points=grid * grid * 150,  # 15000, #args.n_points,
        n_samples=10001,  # args.n_iterations,
        grid_res=256,  # args.grid_res,
        grid_range=1.1,  # args.grid_range,
        sample_type="uniform_central_gaussian",  # args.nonmnfld_sample_type,
        sampling_std=0.5,  # args.nonmnfld_sample_std,
        n_random_samples=7500,  # args.n_random_samples,
        resample=True,
        compute_sal_dist_gt=(True if "sal" in loss_type and loss_weights[5] > 0 else False),
        scale_method="mean",  # "mean" #args.pcd_scale_method,
    )

    model = Net.Network(
        latent_size=0,  # args.latent_size,
        in_dim=3,
        decoder_hidden_dim=128,  # args.decoder_hidden_dim,
        nl="sine",  # args.nl,
        encoder_type="none",  # args.encoder_type,
        decoder_n_hidden_layers=5,  # args.decoder_n_hidden_layers,
        neuron_type="quadratic",  # args.neuron_type,
        init_type="mfgi",  # args.init_type,
        sphere_init_params=[1.6, 0.1],  # args.sphere_init_params,
        n_repeat_period=30,  # args.n_repeat_period,
    )
    model.to(device)

    ######
    test_dataloader = torch.utils.data.DataLoader(
        train_set, batch_size=1, shuffle=False, num_workers=0, pin_memory=False
    )
    test_data = next(iter(test_dataloader))
    mnfld_points = test_data["mnfld_points"].to(device)

    # add noise to mnfld_points
    # mnfld_points += torch.randn_like(mnfld_points) * noise_scale * 2

    mnfld_points.requires_grad_()
    print("mnfld_points shape: ", mnfld_points.shape)
    if torch.cuda.is_available():
        map_location = torch.device("cuda")
    else:
        map_location = torch.device("cpu")
    model.load_state_dict(torch.load(trained_model_path, weights_only=True, map_location=map_location))
    sdf0 = model(sites)

else:

    def sphere_sdf(points: torch.Tensor, center: torch.Tensor, radius: float) -> torch.Tensor:
        """
        Compute the SDF of a sphere at given 3D points.

        Args:
            points: (N, 3) tensor of 3D query points
            center: (3,) tensor specifying the center of the sphere
            radius: float, radius of the sphere

        Returns:
            sdf: (N,) tensor of signed distances
        """
        return torch.norm(points - center, dim=-1) - radius

    def sphere_sdf_with_noise(
        points: torch.Tensor, center: torch.Tensor, radius: float, noise_amplitude=0.05
    ) -> torch.Tensor:
        """
        Sphere SDF with smooth directional noise added near the surface.

        Args:
            points: (N, 3)
            center: (3,)
            radius: float
            noise_amplitude: float

        Returns:
            sdf: (N,)
        """
        rel = points - center
        norm = torch.norm(rel, dim=-1)  # (N,)
        base_sdf = norm - radius  # (N,)

        # Smooth periodic noise based on direction
        unit_dir = rel / (norm.unsqueeze(-1) + 1e-9)  # (N,3)
        noise = torch.sin(10 * unit_dir[:, 0]) * torch.sin(10 * unit_dir[:, 1]) * torch.sin(10 * unit_dir[:, 2])

        # Weight noise so it mostly affects surface area
        falloff = torch.exp(-20 * (base_sdf**2))  # (N,) ~1 near surface, ~0 far
        sdf = base_sdf + noise_amplitude * noise * falloff

        return sdf

    # generate points on the sphere
    mnfld_points = torch.randn(grid * grid * 150, 3, device=device)
    mnfld_points = mnfld_points / torch.norm(mnfld_points, dim=-1, keepdim=True) * 0.5
    mnfld_points = mnfld_points.unsqueeze(0).requires_grad_()
    sdf0 = sphere_sdf(sites, torch.zeros(3).to(device), 0.50)
    # sdf0 = sphere_sdf_with_noise(sites, torch.zeros(3).to(device), 0.50, noise_amplitude=0.1)

##add mnfld points with random noise to sites
# N = mnfld_points.squeeze(0).shape[0]
# num_samples = 24**3 - (num_centroids)
# idx = torch.randint(0, N, (num_samples,))
# sampled = mnfld_points.squeeze(0)[idx]
# perturbed = sampled + (torch.rand_like(sampled)-0.5)*0.05
# sites = torch.cat((sites, perturbed), dim=0)

# make sites a leaf tensor
sites = sites.detach().requires_grad_()
print(sites.dtype)
print(sites.shape)
print(f"Allocated: {torch.cuda.memory_allocated() / 1e6} MB, Reserved: {torch.cuda.memory_reserved() / 1e6} MB")


sdf0 = sdf0.detach().squeeze(-1).requires_grad_()
print(sdf0.shape)
print(sdf0.is_leaf)


mnfld_points shape:  torch.Size([1, 153600, 3])
torch.float32
torch.Size([32768, 3])
Allocated: 432.168448 MB, Reserved: 448.790528 MB
torch.Size([32768])
True


In [4]:
sites_np = sites.detach().cpu().numpy()
d3dsimplices, _ = pygdel3d.triangulate(sites_np)
d3dsimplices = np.array(d3dsimplices)
print("Delaunay simplices shape: ", d3dsimplices.shape)

print("sites shape: ", sites.shape)


Delaunay simplices shape:  (223649, 4)
sites shape:  torch.Size([32768, 3])


In [5]:
ps_cloud = ps.register_point_cloud("initial_cvt_grid+pc_gt", sites.detach().cpu().numpy(), enabled=False)
ps_cloud.add_scalar_quantity(
    "vis_grid_pred",
    sdf0.detach().cpu().numpy(),
    enabled=True,
    cmap="coolwarm",
    vminmax=(-0.00005, 0.00005),
)
mnf_cloud = ps.register_point_cloud("mnfld_points_pred", mnfld_points.squeeze(0).detach().cpu().numpy(), enabled=False)

v_vect, f_vect, sdf_verts, sdf_verts_grads, _ = su.get_clipped_mesh_numba(sites, None, d3dsimplices, False, sdf0, True)
ps_mesh = ps.register_surface_mesh(
    "sdf unclipped initial mesh",
    v_vect.detach().cpu().numpy(),
    f_vect,
    back_face_policy="identical",
)
# ps_vert = ps.register_point_cloud("sdf unclipped initial verts", v_vect.detach().cpu().numpy(), enabled=False)

v_vect, f_vect, _, _, _ = su.get_clipped_mesh_numba(sites, None, d3dsimplices, True, sdf0, True)
ps_mesh = ps.register_surface_mesh(
    "sdf clipped initial mesh",
    v_vect.detach().cpu().numpy(),
    f_vect,
    back_face_policy="identical",
)

d3dsimplices, _ = pygdel3d.triangulate(sites_np)
d3dsimplices = torch.tensor(d3dsimplices, device=device)
marching_tetrehedra_mesh = kaolin.ops.conversions.marching_tetrahedra(
    sites.unsqueeze(0), d3dsimplices, sdf0.unsqueeze(0), return_tet_idx=False
)
vertices_list, faces_list = marching_tetrehedra_mesh
v_vect = vertices_list[0]
faces = faces_list[0]

ps.register_surface_mesh(
    "init MTET", v_vect.detach().cpu().numpy(), faces.detach().cpu().numpy(), back_face_policy="identical"
)


# ps_cloud = ps.register_point_cloud("active sites", tet_probs[2].reshape(-1, 3).detach().cpu().numpy(), enabled=False)
# ps_cloud.add_vector_quantity("site step dir", tet_probs[0].reshape(-1, 3).detach().cpu().numpy())
# ps_vert.add_vector_quantity("verts step dir", tet_probs[1].detach().cpu().numpy())


ps.show()

/home/wylliam/dev/Kyushu_experiments-1/sdfpred_utils/sdfpred_utils.py:1565: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at /pytorch/aten/src/ATen/native/Cross.cpp:62.)
  n = torch.cross(bd, cd)


In [6]:
# SITES OPTIMISATION LOOP


cvt_loss_values = []
min_distance_loss_values = []
chamfer_distance_loss_values = []
eikonal_loss_values = []
domain_restriction_loss_values = []
sdf_loss_values = []
div_loss_values = []
loss_values = []

voroloss = lf.Voroloss_opt().to(device)


def train_DCCVT(
    sites,
    sites_sdf,
    max_iter=100,
    stop_train_threshold=1e-6,
    upsampling=0,
    lambda_weights=[0.1, 1.0, 0.1, 0.1, 1.0, 1.0, 0.1],
    voroloss_optim=False,
):
    if not voroloss_optim:
        optimizer = torch.optim.Adam(
            [
                {"params": [sites], "lr": lr_sites * 0.1},
                {"params": [sites_sdf], "lr": lr_sites * 0.1},
            ]
        )
    else:
        optimizer = torch.optim.Adam([{"params": [sites], "lr": lr_sites * 0.1}])

    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.999)

    # optimizer_sites = torch.optim.Adam([{'params': [sites], 'lr': lr_sites}])
    # optimizer_sdf = torch.optim.SGD([{'params': [sites_sdf], 'lr': lr_sites}])
    # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80, 150, 200, 250], gamma=0.5)

    prev_loss = float("inf")
    best_loss = float("inf")
    upsampled = 0.0
    epoch = 0
    lambda_cvt = lambda_weights[0]
    lambda_chamfer = lambda_weights[4]
    best_sites = sites.clone()
    best_sites.best_loss = best_loss

    while epoch <= max_iter:
        if epoch == max_iter * 0.5:
            print("Halfway through training...")
            optimizer.param_groups[0]["lr"] *= 0.0  # Reduce learning rate to zero for the position
            optimizer.param_groups[1]["lr"] *= 1.0  # Reduce learning rate for the SDF
        optimizer.zero_grad()
        # if mesh[0] == "sphere":
        #     # generate sphere sdf
        #     sites_sdf = sphere_sdf(sites, torch.zeros(3).to(device), 0.50)

        if not voroloss_optim:
            sites_np = sites.detach().cpu().numpy()
            # d3dsimplices = diffvoronoi.get_delaunay_simplices(sites_np.reshape(input_dims * sites_np.shape[0]))
            d3dsimplices, _ = pygdel3d.triangulate(sites_np)

            d3dsimplices = np.array(d3dsimplices)

            cvt_loss = lf.compute_cvt_loss_vectorized_delaunay(sites, None, d3dsimplices)  # torch.tensor(0)  #

            build_mesh = False
            clip = True
            mtet = False
            sites_sdf_grads = None

            if mtet:
                print("Using MTET")
                d3dsimplices = torch.tensor(d3dsimplices, device=device)
                marching_tetrehedra_mesh = kaolin.ops.conversions.marching_tetrahedra(
                    sites.unsqueeze(0), d3dsimplices, sites_sdf.unsqueeze(0), return_tet_idx=False
                )
                vertices_list, faces_list = marching_tetrehedra_mesh
                v_vect = vertices_list[0]
                faces = faces_list[0]
                print("v_vect shape: ", v_vect.shape)

            else:
                v_vect, f_vect, sites_sdf_grads, _, _ = su.get_clipped_mesh_numba(
                    sites, None, d3dsimplices, clip, sites_sdf, build_mesh
                )

            if build_mesh:
                triangle_faces = [[f[0], f[i], f[i + 1]] for f in f_vect for i in range(1, len(f) - 1)]
                triangle_faces = torch.tensor(triangle_faces, device=device)
                hs_p = su.sample_mesh_points_heitz(v_vect, triangle_faces, num_samples=mnfld_points.shape[0])
                chamfer_loss_mesh, _ = chamfer_distance(mnfld_points.detach(), hs_p.unsqueeze(0))
            else:
                chamfer_loss_mesh, _ = chamfer_distance(mnfld_points.detach(), v_vect.unsqueeze(0))

            print(
                "cvt_loss: ",
                lambda_cvt / 10 * cvt_loss.item(),
                "chamfer_loss_mesh: ",
                lambda_chamfer * chamfer_loss_mesh.item(),
            )
            sites_loss = lambda_cvt / 10 * cvt_loss + lambda_chamfer * chamfer_loss_mesh

            if sites_sdf_grads is None:
                sites_sdf_grads = su.sdf_space_grad_pytorch_diego(
                    sites, sites_sdf, torch.tensor(d3dsimplices).to(device).detach()
                )

            eik_loss = lambda_cvt / 10 * lf.discrete_tet_volume_eikonal_loss(sites, sites_sdf_grads, d3dsimplices)
            print("eikonal_loss: ", eik_loss.item(), "motion_loss: ")
            shl = lambda_cvt / 0.1 * lf.smoothed_heaviside_loss(sites, sites_sdf, sites_sdf_grads, d3dsimplices)
            print("smoothed_heaviside_loss: ", shl.item())
            sdf_loss = eik_loss + shl
        else:
            sites_loss = lambda_chamfer * voroloss(mnfld_points.squeeze(0), sites).mean()

        loss = sites_loss + sdf_loss
        loss_values.append(loss.item())
        print(f"Epoch {epoch}: loss = {loss.item()}")

        # print(f"before loss.backward(): Allocated: {torch.cuda.memory_allocated() / 1e6} MB, Reserved: {torch.cuda.memory_reserved() / 1e6} MB")
        loss.backward()
        # print(f"After loss.backward(): Allocated: {torch.cuda.memory_allocated() / 1e6} MB, Reserved: {torch.cuda.memory_reserved() / 1e6} MB")
        print("-----------------")

        # torch.nn.utils.clip_grad_norm_(sites_sdf, 1.0)
        # torch.nn.utils.clip_grad_norm_(sites, 1.0)
        optimizer.step()

        # sites_sdf += (sites_sdf_grads*(sites-sites_positions)).sum(dim=1)

        # scheduler.step()
        print("Learning rate: ", optimizer.param_groups[0]["lr"])
        # if epoch>100 and (epoch // 100) == upsampled+1 and loss.item() < 0.5 and upsampled < upsampling:

        if upsampled < upsampling and epoch / (max_iter * 0.80) > upsampled / upsampling:
            print("sites length BEFORE UPSAMPLING: ", len(sites))
            if len(sites) * 1.09 > grid**3:
                print("Skipping upsampling, too many sites, sites length: ", len(sites), "grid size: ", grid**3)
                upsampled = upsampling
                sites = sites.detach().requires_grad_(True)
                sites_sdf = sites_sdf.detach().requires_grad_(True)

                optimizer = torch.optim.Adam(
                    [
                        {"params": [sites], "lr": lr_sites * 0.1},
                        {"params": [sites_sdf], "lr": lr_sites * 0.1},
                    ]
                )
                # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)
                continue
            # sites, sites_sdf = su.upsampling_vectorized_sites_sites_sdf(sites, tri=None, vor=None, simplices=d3dsimplices, model=sites_sdf)
            # sites, sites_sdf = su.upsampling_curvature_vectorized_sites_sites_sdf(sites, tri=None, vor=None, simplices=d3dsimplices, model=sites_sdf)
            sites, sites_sdf = su.upsampling_adaptive_vectorized_sites_sites_sdf(
                sites, simplices=d3dsimplices, model=sites_sdf
            )

            # sites, sites_sdf = su.upsampling_chamfer_vectorized_sites_sites_sdf(
            #     sites, d3dsimplices, sites_sdf, mnfld_points
            # )

            sites = sites.detach().requires_grad_(True)
            sites_sdf = sites_sdf.detach().requires_grad_(True)

            optimizer = torch.optim.Adam(
                [
                    {"params": [sites], "lr": lr_sites * 0.1},
                    {"params": [sites_sdf], "lr": lr_sites * 0.1},
                ]
            )
            # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)

            upsampled += 1.0
            print("sites shape AFTER: ", sites.shape)
            print("sites sdf shape AFTER: ", sites_sdf.shape)

        if epoch % (max_iter / 10) == 0 or epoch == max_iter:
            # print(f"Epoch {epoch}: loss = {loss.item()}")
            # print(f"Best Epoch {best_epoch}: Best loss = {best_loss}")
            # save model and sites
            # ps.register_surface_mesh(f"{epoch} triangle clipped mesh", v_vect.detach().cpu().numpy(), triangle_faces.detach().cpu().numpy())

            # ps.register_point_cloud('sampled points end', hs_p.detach().cpu().numpy())
            # ps.register_point_cloud("sampled points end", v_vect.detach().cpu().numpy(), enabled=False)

            # if f_vect is not None:
            #     ps_mesh = ps.register_surface_mesh(
            #         f"{epoch} sdf clipped pmesh",
            #         v_vect.detach().cpu().numpy(),
            #         f_vect,
            #         back_face_policy="identical",
            #         enabled=False,
            #     )
            #     ps_mesh.add_vector_quantity(
            #         f"{epoch} sdf verts grads",
            #         sdf_verts_grads.detach().cpu().numpy(),
            #         enabled=False,
            #     )

            site_file_path = (
                f"{destination}{mesh[0]}{max_iter}_{epoch}_3d_sites_{num_centroids}_chamfer{lambda_chamfer}.pth"
            )
            # model_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_model_{num_centroids}_chamfer{lambda_chamfer}.pth'
            sdf_file_path = (
                f"{destination}{mesh[0]}{max_iter}_{epoch}_3d_sdf_{num_centroids}_chamfer{lambda_chamfer}.pth"
            )
            torch.save(sites_sdf, sdf_file_path)
            torch.save(sites, site_file_path)

        epoch += 1

    return sites, sites_sdf

In [7]:
# lambda_weights = [252,0,0,0,10.211111,0,100,0]
# lambda_weights = [500,0,0,0,1000,0,100,0]
lambda_weights = [100, 0, 0, 0, 1000, 0, 100, 0]


lambda_cvt = lambda_weights[0]
lambda_sdf = lambda_weights[1]
lambda_min_distance = lambda_weights[2]
lambda_laplace = lambda_weights[3]
lambda_chamfer = lambda_weights[4]
lambda_eikonal = lambda_weights[5]
lambda_domain_restriction = lambda_weights[6]
lambda_true_points = lambda_weights[7]

max_iter = 1000

In [8]:
site_file_path = f"{destination}{max_iter}_cvt_{lambda_cvt}_chamfer_{lambda_chamfer}_eikonal_{lambda_eikonal}.npy"
# check if optimized sites file exists
if not os.path.exists(site_file_path):
    # import sites
    print("Importing sites")
    sites = np.load(site_file_path)
    sites = torch.from_numpy(sites).to(device).requires_grad_(True)
else:
    # import cProfile, pstats
    # import time

    # profiler = cProfile.Profile()
    # profiler.enable()

    # with torch.profiler.profile(
    #     activities=[
    #         torch.profiler.ProfilerActivity.CPU,
    #         torch.profiler.ProfilerActivity.CUDA,
    #     ],
    #     record_shapes=False,
    #     with_stack=True,  # Captures function calls
    # ) as prof:
    #     sites, optimized_sites_sdf = train_DCCVT(
    #         sites, sdf0, max_iter=max_iter, upsampling=0, lambda_weights=lambda_weights
    #     )

    # print(prof.key_averages().table(sort_by="self_cuda_time_total"))
    # prof.export_chrome_trace("trace.json")

    sites, optimized_sites_sdf = train_DCCVT(
        sites, sdf0, max_iter=max_iter, upsampling=0, lambda_weights=lambda_weights
    )

    sites_np = sites.detach().cpu().numpy()
    np.save(site_file_path, sites_np)

print("Sites length: ", len(sites))
print("min sites: ", torch.min(sites))
print("max sites: ", torch.max(sites))

cvt_loss:  0.9514137357473373 chamfer_loss_mesh:  0.4217838286422193
eikonal_loss:  0.9358572363853455 motion_loss: 
smoothed_heaviside_loss:  0.13841313123703003
Epoch 0: loss = 2.447467803955078
-----------------
Learning rate:  0.0005
cvt_loss:  0.8154639601707458 chamfer_loss_mesh:  0.39039310649968684
eikonal_loss:  0.5509101152420044 motion_loss: 
smoothed_heaviside_loss:  0.1385490447282791
Epoch 1: loss = 1.8953161239624023
-----------------
Learning rate:  0.0005
cvt_loss:  0.8296500891447067 chamfer_loss_mesh:  0.3701582900248468
eikonal_loss:  0.3527001738548279 motion_loss: 
smoothed_heaviside_loss:  0.13891759514808655
Epoch 2: loss = 1.691426157951355
-----------------
Learning rate:  0.0005
cvt_loss:  0.8620405197143555 chamfer_loss_mesh:  0.35210861824452877
eikonal_loss:  0.24081037938594818 motion_loss: 
smoothed_heaviside_loss:  0.13926947116851807
Epoch 3: loss = 1.594228982925415
-----------------
Learning rate:  0.0005
cvt_loss:  0.84367536008358 chamfer_loss_mesh

In [9]:
epoch = 1000

# model_file_path = f'{destination}{mesh[0]}{max_iter}_{epoch}_3d_model_{num_centroids}_chamfer{lambda_chamfer}.pth'
site_file_path = f"{destination}{mesh[0]}{max_iter}_{epoch}_3d_sites_{num_centroids}_chamfer{lambda_chamfer}.pth"
sdf_file_path = f"{destination}{mesh[0]}{max_iter}_{epoch}_3d_sdf_{num_centroids}_chamfer{lambda_chamfer}.pth"


sites = torch.load(site_file_path)
sdf_v = torch.load(sdf_file_path)

sites_np = sites.detach().cpu().numpy()
print("sdf", sdf_v.shape)
print("sites", site_file_path)

ps_cloud_f = ps.register_point_cloud(f"{epoch} epoch_cvt_grid", sites_np)
ps_cloud_f.add_scalar_quantity(
    "vis_grid_pred",
    sdf_v.detach().cpu().numpy(),
    enabled=True,
    cmap="coolwarm",
    vminmax=(-0.15, 0.15),
)

print("sites_np shape: ", sites_np.shape)

# print sites if Nan
if np.isnan(sites_np).any():
    print("sites_np contains NaN values")
    print("sites_np NaN values: ", np.isnan(sites_np).sum())
# remove nan values from sites tensor
sites_np = sites_np[~np.isnan(sites_np).any(axis=1)]
sites = torch.from_numpy(sites_np).to(device).requires_grad_(True)

sdf torch.Size([32768])
sites ./images/autograd/End2End_DCCVT_interpolSDF/gargoyle1000_1000_3d_sites_32768_chamfer1000.pth
sites_np shape:  (32768, 3)


In [10]:
# metric between sites sdf values and their corresponding sdf values on hotspot model
true_Sdf = model(sites).squeeze(-1)
d3dsimplices, _ = pygdel3d.triangulate(sites_np)
d3dsimplices = np.array(d3dsimplices)
vertices_to_compute, bisectors_to_compute, used_tet = su.compute_zero_crossing_vertices_3d(
    sites, None, None, d3dsimplices, sdf_v
)
d3dsimplices = torch.tensor(d3dsimplices, device=device)
d3d = d3dsimplices[used_tet]
zc_sdf = sdf_v[d3d]
zc_truesdf = true_Sdf[d3d]
print("zc true_Sdf shape: ", zc_truesdf.shape)
print("zc optimized sdf :", zc_sdf.shape)
print("sum of zc true Sdf: ", torch.sum(zc_truesdf).item())
print("sum of zc opti Sdf: ", torch.sum(zc_sdf).item())
print("Diff   of   sum: ", torch.sum(zc_truesdf - zc_sdf).item())
print("Mean of zc true Sdf: ", torch.mean(zc_truesdf - zc_sdf).item())

zc true_Sdf shape:  torch.Size([7199, 4])
zc optimized sdf : torch.Size([7199, 4])
sum of zc true Sdf:  499.7693786621094
sum of zc opti Sdf:  421.29296875
Diff   of   sum:  78.4764175415039
Mean of zc true Sdf:  0.0027252542786300182


In [11]:
# v_vect, f_vect = su.get_clipped_mesh_numba(sites, model, None, True)
# ps.register_surface_mesh("model final clipped polygon mesh", v_vect.detach().cpu().numpy(), f_vect)

# v_vect, f_vect = su.get_clipped_mesh_numba(sites, model, None, False)
# ps.register_surface_mesh("model final polygon mesh", v_vect.detach().cpu().numpy(), f_vect)

######################################################

# if mesh[0] == "sphere":
#     # generate sphere sdf
#     print("Generating sphere SDF")
#     sdf_v = sphere_sdf(sites, torch.zeros(3).to(device), 0.50)


(
    v_vect,
    f_vect,
    _,
    _,
    _,
) = su.get_clipped_mesh_numba(sites, None, None, False, sdf_v, True)

f_vect = [[f[0], f[i], f[i + 1]] for f in f_vect for i in range(1, len(f) - 1)]

ps.register_surface_mesh("sdf final unclipped polygon mesh", v_vect.detach().cpu().numpy(), f_vect)


v_vect, f_vect, _, _, _ = su.get_clipped_mesh_numba(sites, None, None, True, sdf_v, True)
f_vect = [[f[0], f[i], f[i + 1]] for f in f_vect for i in range(1, len(f) - 1)]
ps.register_surface_mesh("sdf final clipped polygon mesh", v_vect.detach().cpu().numpy(), f_vect)
# f_vect = [[f[0], f[i], f[i + 1]] for f in f_vect for i in range(1, len(f) - 1)]

d3dsimplices, _ = pygdel3d.triangulate(sites_np)
d3dsimplices = torch.tensor(d3dsimplices, device=device)
marching_tetrehedra_mesh = kaolin.ops.conversions.marching_tetrahedra(
    sites.unsqueeze(0), d3dsimplices, sdf_v.unsqueeze(0), return_tet_idx=False
)
vertices_list, faces_list = marching_tetrehedra_mesh
v_vect = vertices_list[0]
faces = faces_list[0]

ps.register_surface_mesh(
    "MTET", v_vect.detach().cpu().numpy(), faces.detach().cpu().numpy(), back_face_policy="identical"
)

# export obj file
output_obj_file = (
    f"{destination}{mesh[0]}{max_iter}_{epoch}_3d_sites_{num_centroids}_chamfer{lambda_chamfer}_outputmesh.obj"
)
output_ply_file = (
    f"{destination}{mesh[0]}{max_iter}_{epoch}_3d_sites_{num_centroids}_chamfer{lambda_chamfer}_targetpointcloud.ply"
)
su.save_obj(output_obj_file, v_vect.detach().cpu().numpy(), f_vect)
su.save_target_pc_ply(output_ply_file, mnfld_points.squeeze(0).detach().cpu().numpy())

ps.show()

Computing Delaunay simplices...
Number of Delaunay simplices: 202064
Delaunay simplices shape: [[ 1024  4096  2080  2048]
 [ 1059  3072  1024  1025]
 [ 1056  1024     0  4096]
 ...
 [ 9686  9655 10710  8629]
 [ 9686 10709  9685  8629]
 [ 9686 10709 10710  9685]]
Max vertex index in simplices: 32767
Min vertex index in simplices: 0
Site index range: 32768
Computing Delaunay simplices...
Number of Delaunay simplices: 202064
Delaunay simplices shape: [[ 1059  3072  1024  1025]
 [12657 12656 11601 10577]
 [   34  1059     0     1]
 ...
 [ 9686  9655 10710  8629]
 [ 9686 10709  9685  8629]
 [ 9686 10709 10710  9685]]
Max vertex index in simplices: 32767
Min vertex index in simplices: 0
Site index range: 32768


In [12]:
# sites, sdf = train_DCCVT(
#     sites, sdf_v, max_iter=max_iter, upsampling=0, lambda_weights=lambda_weights, voroloss_optim=True
# )
# (
#     v_vect,
#     f_vect,
#     _,
#     _,
#     _,
# ) = su.get_clipped_mesh_numba(sites, None, None, False, sdf, True)
# ps.register_surface_mesh("voromeh sdf final unclipped polygon mesh", v_vect.detach().cpu().numpy(), f_vect)


# v_vect, f_vect, _, _, _ = su.get_clipped_mesh_numba(sites, None, None, True, sdf, True)
# ps.register_surface_mesh("voromeh sdf final clipped polygon mesh", v_vect.detach().cpu().numpy(), f_vect)
# # f_vect = [[f[0], f[i], f[i + 1]] for f in f_vect for i in range(1, len(f) - 1)]
# ps.show()


In [13]:
# chamfer metric
# add sampled points to polyscope and ground truth mesh to polyscope

import trimesh


def sample_points_on_mesh(mesh_path, n_points=100000):
    mesh = trimesh.load(mesh_path)
    # normalize mesh
    mesh.apply_translation(-mesh.centroid)
    mesh.apply_scale(1.0 / np.max(np.abs(mesh.vertices)))
    # export mesh to obj file
    mesh.export(mesh_path.replace(".obj", ".obj"))
    print(mesh_path)
    points, _ = trimesh.sample.sample_surface(mesh, n_points)
    return points, mesh


import numpy as np
from scipy.spatial import cKDTree


def chamfer_accuracy_completeness(ours_pts, gt_pts):
    # Completeness: GT → Ours
    dists_gt_to_ours = cKDTree(ours_pts).query(gt_pts, k=1)[0]
    completeness = np.mean(dists_gt_to_ours**2)

    # Accuracy: Ours → GT
    dists_ours_to_gt = cKDTree(gt_pts).query(ours_pts, k=1)[0]
    accuracy = np.mean(dists_ours_to_gt**2)

    return accuracy, completeness


ours_pts, _ = sample_points_on_mesh(output_obj_file, n_points=100000)
m = mesh[1].replace("data", "mesh")
gt_pts, _ = sample_points_on_mesh(m + ".obj", n_points=100000)

acc, comp = chamfer_accuracy_completeness(ours_pts, gt_pts)

print(f"Chamfer Accuracy (Ours → GT): {acc:.6f}")
print(f"Chamfer Completeness (GT → Ours): {comp:.6f}")
print(f"Chamfer Distance (symmetric): {acc + comp:.6f}")


IndexError: index 5284 is out of bounds for axis 0 with size 4719

In [ ]:
def sample_points_on_mesh(mesh_path, n_points=100000):
    mesh = trimesh.load(mesh_path)

    # Normalize mesh (centered and scaled uniformly)
    bbox = mesh.bounds
    center = mesh.centroid
    scale = np.linalg.norm(bbox[1] - bbox[0])
    mesh.apply_translation(-center)
    mesh.apply_scale(1.0 / scale)

    # Export normalized mesh
    mesh.export(mesh_path.replace(".obj", ".obj"))

    points, _ = trimesh.sample.sample_surface(mesh, n_points)
    return points, mesh


_, _ = sample_points_on_mesh(
    "/home/wylliam/dev/Kyushu_experiments/outputs/gargoyle_unconverged/cdp1000_v0_cvt100_clipTrue_buildFalse_upsampling0_num_centroids32_target_size32_final.obj",
    n_points=100000,
)